In [1]:
from huggingface_hub import login
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/vikas/.cache/huggingface/token
Login successful


In [2]:
path="DocBot/english/"

In [3]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(
    filename=path + "sample_pdf.pdf",
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=1850,
    new_after_n_chars=1750,
    combine_text_under_n_chars=1000,
    image_output_dir_path=path,
)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [4]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 13,
 "<class 'unstructured.documents.elements.Table'>": 1}

In [5]:
raw_pdf_elements[0].metadata.orig_elements[1].to_dict()

{'type': 'NarrativeText',
 'element_id': '96a8ec9a-845a-45b5-8e27-759643bdc150',
 'text': 'YRC (Yamaha Ride Control) Yamaha Ride Control is a system that incorporates numerous sensors and controls to support an improved riding experience. The vehicle senses and can react to forces along the longitudi- nal (front-to-back), lateral (left-to-right), and vertical (up-and-down) axes. Lean angle and G-force accelerations are also detected. This information is proc- essed multiple times a second and the related physical systems are automati- cally adjusted as necessary. The fol- lowing functions represent individual YRC items which can be turned on/off or adjusted to suit various riders and riding conditions. For setting details, see pages 4-15 and 4-19.',
 'metadata': {'detection_class_prob': 0.9282627701759338,
  'coordinates': {'points': ((173.21183333333323, 229.26394444444438),
    (173.21183333333323, 829.2650555555554),
    (647.74169921875, 829.2650555555554),
    (647.74169921875, 22

In [6]:
unchunked_list = []

for ele in raw_pdf_elements:
    sublist = []
    for orig_ele in ele.metadata.orig_elements:
        metadata = orig_ele.metadata.to_dict()
        filtered_dict = {}
        filtered_dict['text'] = orig_ele.to_dict()['text']
        filtered_dict['type'] = orig_ele.to_dict()['type']
        filtered_dict['page_number'] = metadata['page_number']
        if 'image_path' in metadata:
            filtered_dict['image_path']=metadata['image_path']
        sublist.append(filtered_dict)
    unchunked_list.append(sublist)
print(len(unchunked_list))

chunk_text_list = [i.text for i in raw_pdf_elements]
chunk_text_list[3]

14


'EWaz08e0\n\nEWA20880\n\nMake sure the engine has sufficient- ly slowed before shifting to a lower gear. Engaging a lower gear when the engine speed is too high could make the rear wheel lose traction. This could cause loss of control, an accident and injury. It could also cause engine or drivetrain damage.\n\nQuick shifter The quick shifter allows for clutch lev- er-less, electronically-assisted shifting.\n\n—p—\n\nWhen the sensor on the shift rod de- tects the appropriate motion in the shift pedal, engine power output is momen- tarily adjusted to allow for the gear change to occur. The quick shifter does not operate when the clutch lever is pulled, there- fore normal shifting can be done even when the quick shifter is set to on. Check the quick shifter indicator for current status and usability informa- tion.'

In [7]:
unchunked_list[0][1]
# raw_pdf_elements[0].metadata.orig_elements[10].to_dict()

{'text': 'YRC (Yamaha Ride Control) Yamaha Ride Control is a system that incorporates numerous sensors and controls to support an improved riding experience. The vehicle senses and can react to forces along the longitudi- nal (front-to-back), lateral (left-to-right), and vertical (up-and-down) axes. Lean angle and G-force accelerations are also detected. This information is proc- essed multiple times a second and the related physical systems are automati- cally adjusted as necessary. The fol- lowing functions represent individual YRC items which can be turned on/off or adjusted to suit various riders and riding conditions. For setting details, see pages 4-15 and 4-19.',
 'type': 'NarrativeText',
 'page_number': 1}

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
# Used to securely store your API key

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from ctransformers import AutoModelForCausalLM as cAutoModelForCausalLM
from ctransformers import AutoConfig, Config

In [3]:
conf = AutoConfig(Config(temperature=0.8, repetition_penalty=1.1,
                         batch_size=52, max_new_tokens=3000,
                         context_length=4096,gpu_layers=5))
llm = cAutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GGUF", model_type="mistral", config=conf)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
def generate_output(question):
    formatted_input = f"<s> <INST> {question} </INST>"
    output = llm(formatted_input,repetition_penalty=1.15, max_new_tokens=3000)

    return output

def prompt_input(input_text):
    prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. \
The text chunk which you will receive will be in the following format: {{'text': '', 'type': '', 'page_number': }}. If the type argument has value 'Title', THEN MAKE SURE to include it in your summary. There may be more than one title in the text chunk you will receive; make sure that you give EACH text which has 'Title' type in your summary output. If there is any text in the image, include that too. Ignore all the page numbers. Do not use new line characters. \
Table or text chunk: {element} """
    return prompt_text.format(element=input_text)

In [11]:
lambda_chain = lambda input_list: list(map(lambda text: generate_output(prompt_input(text)), input_list))
# Apply to text
text_summaries = lambda_chain(unchunked_list)

In [12]:
text_summaries

[' Title: Yamaha Ride Control (YRC); this system enhances riding experience by incorporating sensors and adjusting vehicle responses based on forces along longitudinal, lateral, and vertical axes. The system includes functions that can be adjusted or turned on/off to suit riders and conditions. Warning: YRC is not a substitute for proper riding techniques or expertise; it cannot prevent loss of control caused by rider errors such as excessive speed or sharp turns, or hard braking. Power delivery mode system offers four control maps allowing adjustment of throttle valve opening based on preferred settings and environment.',
 ' This text and image document discusses the traction control system in a Yamaha vehicle. The system helps maintain traction by regulating engine power when sensors detect slippage in the rear wheel. When cornering, it applies more control to maximize acceleration. There are tips provided regarding its engagement over bumps and noticeable changes in sounds when enga

In [8]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries", embedding_function=GPT4AllEmbeddings(), persist_directory="./chromadb"
)

# The storage layer for the parent documents
store = InMemoryStore()  # <- Can we extend this to images
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [13]:
# store.mdelete(list(store.__dict__['store'].keys()))
# vectorstore.delete(vectorstore.get()['ids'])
print(list(store.__dict__['store'].keys()))
print(vectorstore.get()['ids'])

['22b0678a-61f7-45dd-af3f-2c65b16a9598', '978f23c5-4ccd-45fa-a153-caeea2052f8f', 'da7b77a9-edef-4b52-8879-c66b9d530ac7', '7bdf332a-9206-418e-b8c6-c07879e2691b', 'db775636-ace4-4312-a2ee-abd65d507644', 'dc7291e9-9401-46f0-9691-4d22bb380ffb', '834d93d4-005f-4808-8c50-a8b193d38f56', '0673da4b-67da-4b5c-8eb9-401384573bb4', '2e872210-1157-4064-b09c-177895bc65cb', 'f14f2486-df69-402f-93ed-06e6dbb14c1f', '97372640-1f99-4b4e-b774-d8ed655c364c', 'fca97113-e891-41fe-94af-620291f1f35b', '77ae2b71-0e4a-42a9-aabd-b3f9cf094ee4', '9aec483d-a79e-4c39-b311-818c1ef430ec', 'ab3ad65b-b927-43b2-9923-36ee0b746878', '0eadd4e0-2a96-4697-a47e-8ce197c9f79b', 'cde11df6-7d59-4a5c-8fe0-7f3cd1c76bb8', '3db24b73-9475-47ba-883d-7fbb6db3ad65', 'fccf5c93-8ea2-4c26-b3c0-5290e2850600', '14e8673f-af73-4c5e-8a0b-a51af9e1276c', '3d22a828-26e4-4d3f-b0f0-ead7b24d4be8', '9cbaa180-b622-4ed2-9029-122e1ed99b7b', '9feeb893-7e52-41cb-9c89-6e1954c27866', '3b51ae27-b165-4f34-8d2b-1765cb0fc3be', 'f86bb3fc-3a62-4bce-a8e0-46f2e826bd1a',

In [17]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in text_summaries]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]

In [34]:
# retriever.vectorstore.add_documents(summary_texts)
# retriever.docstore.mset(list(zip(doc_ids, unchunked_list)))

In [4]:
import pickle

def save_to_pickle(obj, filename):
    with open(filename, 'wb') as filehandler:
        pickle.dump(obj, filehandler)

def load_from_pickle(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)


In [5]:
import os
directory = "./datafiles"
os.makedirs(directory, exist_ok=True)
save_path_summary=os.path.join(directory, "summary_texts.pkl")
# save_to_pickle(summary_texts, save_path_summary)
save_path_docstore_data=os.path.join(directory, "docstore_data.pkl")
# save_to_pickle(list(zip(doc_ids, unchunked_list)), save_path_docstore_data)

In [6]:
summary_texts=load_from_pickle(save_path_summary)
save_path_docstore_data=load_from_pickle(save_path_docstore_data)

In [12]:
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(save_path_docstore_data)

In [38]:
user_q="Give me the best feature of Yamaha bike?"
retrieved_query_vectorstore=retriever.vectorstore.similarity_search(user_q)

In [39]:
doc_id1=retrieved_query_vectorstore[0].__dict__['metadata']['doc_id']
doc_id2=retrieved_query_vectorstore[1].__dict__['metadata']['doc_id']
doc_id3=retrieved_query_vectorstore[2].__dict__['metadata']['doc_id']

In [41]:
retriever.docstore.store[doc_id1]

[{'text': '(ae 4', 'type': 'Title', 'page_number': 5},
 {'text': 'DP,', 'type': 'UncategorizedText', 'page_number': 6},
 {'text': 'i i) a', 'type': 'Title', 'page_number': 6},
 {'text': '+', 'type': 'UncategorizedText', 'page_number': 6},
 {'text': 'UBMPE1E0.book Page 5 Monday, August 7, 2023 9:58 AM',
  'type': 'Title',
  'page_number': 6},
 {'text': '“>', 'type': 'UncategorizedText', 'page_number': 6},
 {'text': '$1 @',
  'type': 'Image',
  'page_number': 6,
  'image_path': '/mnt/c/Users/hryad/Desktop/iitm/Knowledge Graph LLM/DocBot_Yamaha/figures/figure-6-3.jpg'},
 {'text': 'WY', 'type': 'Title', 'page_number': 6},
 {'text': 'UBMPE1E0.book Page 1 Monday, August 7, 2023 9:58 AM',
  'type': 'Header',
  'page_number': 7},
 {'text': 'Introduction', 'type': 'Header', 'page_number': 7},
 {'text': 'EAU10103', 'type': 'NarrativeText', 'page_number': 7},
 {'text': 'Welcome to the Yamaha world of motorcycling! As the owner of the YZF1000 / YZF1000D, you are benefiting from Yamaha’s vast exper

In [42]:
def generate_output(question):
    formatted_input = f"<s> <INST> {question} </INST>"
    output = llm(formatted_input,repetition_penalty=1.15, max_new_tokens=3000)
    return output

# def prompt_input(user_q, retrieved_info1,retrieved_info2, retrieved_info3):
#     prompt_text = """
#         You are a helpful, respectful and honest assistant. The user is asking about the given question.
#         User Question:{user_q}
#         Information1:{retrieved_info1}
#         Information2:{retrieved_info2}
#         Information3:{retrieved_info3}
#         The information provided to you is in the form of a list where each element in the list is a dictionary containing the information having text, page_number, and type.
#         Prompt Explanation for LLMs:
#         Tasks:
#         Task 1: Based on the provided information, generate a concise and accurate response to the user's question.
#         Task 2: If applicable, locate and provide the path to the closest image related to the discussion from the list.
#         Task 3: provide page number from which the text is taken.
#         Instructions:
#         Use the information provided to craft a response that directly addresses the user's question.
#         If there are multiple relevant sections, prioritize the most pertinent ones for the response.
#         Ensure that the response is clear, coherent, and free from ambiguity. Try to answer interactively 
#         If an image is available, provide its path only if it directly relates to the user's question or enhances understanding of the topic. Suppose an image comes after a text that you are using, you have to return its path.
#         give the output in the following json format: output: <your-output>, image_path:<image path for closest image or just none if it is not right after the text you have used>, page_number: <the page number fetched>"""
#     return prompt_text.format(user_q=user_q, retrieved_info1=retrieved_info1,retrieved_info2=retrieved_info2,retrieved_info3=retrieved_info3)


def prompt_input_impg(user_q, retrieved_info1,retrieved_info2, retrieved_info3):
    prompt_text = """
        The user is asking the given question. 
        User Question:{user_q}
        Information1:{retrieved_info1}
        Information2:{retrieved_info2}
        Information3:{retrieved_info3}
        The information provided to you is in the form of a list where each element in the list is a dictionary containing the information having text, page_number, and type.
        Prompt Explanation for LLMs:
        Tasks:
        Task 1: If applicable, locate and provide the path to the closest image to the possible answer to the given question from the information list.
        Task 2: provide page number from which the text of PRE TASK is taken.
        Instructions:
        If an image is available, provide its path only if it directly relates to the user's question or enhances understanding of the topic. Suppose an image comes after a text that you are using, you have to return its path.
        give the output in the following json format: image_path:<list of image path for closest image or just none if it is not right after the text you have used>, page_number: <the page number fetched>.
        You dont need to output anything other than image_path (none if no applicable image found) and LIST of page numbers.
        I DONT WANT ANYTHING OTHER THAN THE JSON DICTIONARY IN THE OUTPUT. Example output 1:
        image_path: [], page_number: []
        Example output 2:
        image_path: [/mnt/c/Users/hryad/Desktop/iitm/Knowledge Graph LLM/DocBot_Yamaha/figures/figure-188-307.jpg], page_number: [188, 312]
        
        """
    return prompt_text.format(user_q=user_q, retrieved_info1=retrieved_info1,retrieved_info2=retrieved_info2,retrieved_info3=retrieved_info3)

def prompt_input_para(user_q, retrieved_info1,retrieved_info2, retrieved_info3):
    prompt_text = """
        You are a helpful, respectful and honest assistant. The user is asking about the given question.
        User Question:{user_q}
        Information1:{retrieved_info1}
        Information2:{retrieved_info2}
        Information3:{retrieved_info3}
        The information provided to you is in the form of a list where each element in the list is a dictionary containing the information having text, page_number, and type.
        Prompt Explanation for LLMs:
        Tasks:
        Task 0: Based on the provided information, generate a concise and accurate response to the user's question.
        Instructions:
        Use the information provided to craft a response that directly addresses the user's question.
        If there are multiple relevant sections, prioritize the most pertinent ones for the response.
        You dont need to output any image or image path, or any page number. Just output the text which answers the user's query on the basis of information provided to you.
        Ensure that the response is clear, coherent, and free from ambiguity. Try to answer interactively.
        """
    return prompt_text.format(user_q=user_q, retrieved_info1=retrieved_info1,retrieved_info2=retrieved_info2,retrieved_info3=retrieved_info3)

In [43]:
# prompt=prompt_input(user_q=user_q, retrieved_info1=retriever.docstore.store[doc_id1],retrieved_info2=retriever.docstore.store[doc_id2])
# output=generate_output(prompt)
prompt_impg=prompt_input_impg(user_q=user_q, retrieved_info1=retriever.docstore.store[doc_id1],retrieved_info2=retriever.docstore.store[doc_id2],retrieved_info3=retriever.docstore.store[doc_id3])
output_impg=generate_output(prompt_impg)

prompt_para=prompt_input_para(user_q=user_q, retrieved_info1=retriever.docstore.store[doc_id1],retrieved_info2=retriever.docstore.store[doc_id2],retrieved_info3=retriever.docstore.store[doc_id3])
output_para=generate_output(prompt_para)

In [44]:
user_q

'Give me the best feature of Yamaha bike?'

In [45]:
output_impg

' Based on the user question "Give me the best feature of Yamaha bike", there is no direct text answer in the provided information that specifically mentions the "best feature" of a Yamaha bike. Therefore, I cannot provide an image path or page number for this question. The output will be:\n        \n        image_path: [], page_number: []'

In [46]:
output_para

'  The best feature of a Yamaha bike can vary depending on specific models, but many riders appreciate the advanced safety systems like the YRC (Yamaha Ride Control) which optimizes power distribution based on road conditions and traction requirements, providing increased confidence and control. Additionally, regular maintenance such as checking engine idling speed and adjusting valve clearance are crucial for optimal performance and longevity of your Yamaha bike. Tires should also be maintained in good condition and adjusted at appropriate times to ensure safety on the road.'

In [53]:
import re
output = """["/mnt/c/Users/hryad/Desktop/iitm/Knowledge Graph LLM/DocBot_Yamaha/figures/figure-188-307.jpg" """
image_path_pattern = r'figure-(\d+-\d+)\.jpg'
image_path_match = re.findall(image_path_pattern, output)
images=[]
if len(image_path_match):
    for ele in image_path_match:
        image_name="figure-"+ele +".jpg"
        images.append(image_name)
page_number_pattern = r'"page_numbers?"\s*:\s*\[([0-9\s,"]+)\]'
page_numbers_match = re.search(page_number_pattern, output)
page_numbers = []
if page_numbers_match:
    numbers_group = page_numbers_match.group(1)
    for num in numbers_group.split(','):
        num = num.strip()
        num = num.strip('"')
        if num.isdigit():
            page_numbers.append(int(num))
page_number_pattern = r'"page_number"\s*:\s*(\d+)'
page_numbers_match = re.search(page_number_pattern, output)
if page_numbers_match:
    numbers_group = page_numbers_match.group(1)
    for num in numbers_group.split(','):
        num = num.strip()
        num = num.strip('"')
        if num.isdigit():
            page_numbers.append(int(num))
print(images)
print(page_numbers)


['figure-188-307.jpg']
[]


In [50]:
output = "/mnt/c/Users/hryad/Desktop/iitm/Knowledge Graph LLM/DocBot_Yamaha/figures/figure-188-307.jpg"
image_path_pattern = r'figure-(\d+-\d+)\.jpg'
image_path_match = re.findall(image_path_pattern, output)
images=[]
if len(image_path_match):
    for ele in image_path_match:
        image_name="figure-"+image_path_match[0]+".jpg"
        images.append(image_name)
page_number_pattern = r'"page_numbers?"\s*:\s*\[([0-9\s,"]+)\]'
page_numbers_match = re.search(page_number_pattern, output)
page_numbers = []
if page_numbers_match:
    numbers_group = page_numbers_match.group(1)
    for num in numbers_group.split(','):
        num = num.strip()
        num = num.strip('"')
        if num.isdigit():
            page_numbers.append(int(num))
print(images,page_numbers)

['figure-188-307.jpg'] []
